In [25]:
cd "C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam"

C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam


In [26]:
# cd C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam

In [27]:
# cd C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam

In [28]:
from squid_lab_quam.components.roots import QuAMSCQ1

machine = QuAMSCQ1()

## Add information

In [29]:
from squid_lab_quam.components.information import Information

information = Information(
    user_name="Jacob Hastrup",
    user_ku_tag="drn455",
    device_name="Chalmers/5q4c_v0",
    fridge_name="meso",
    # data_path=r"\\unicph.domain\groupdir\SCI-NBI-QDev\SQuID Lab Data\Devices\Chalmers 5q4c v1.1\OPX Data",
    # state_path=r"C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests",
    # state_path=r"C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests\state",
    state_path="C:/Users/drn455/Documents/GitHub/SQuID-Lab-quam/squid_lab_quam/tests/state",
)

machine.information = information

information.print_info()

User name: Jacob Hastrup (drn455)
Device name: Chalmers/5q4c_v0
Fridge name: meso


## Add network

In [30]:
from squid_lab_quam.components.network import OPXNetwork, OctaveNetwork

network = OPXNetwork(
    host="10.209.67.168",
    cluster_name="Cluster_1",
    octave_networks={
        "octave1": OctaveNetwork(
            octave_host="10.209.67.174", octave_port=80, controller="con1"
        )
    },
)

machine.network = network

## Add wiring

In [31]:
from squid_lab_quam.components.wiring import (
    OPXWiring,
    OPXFeedLineWiring,
    OPXIQChannelWiring,
)

wiring = OPXWiring(
    drive_lines={
        "q0": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q1": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q2": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q3": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q4": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
    },
    feed_lines={
        "feedline": OPXFeedLineWiring(
            output_I=("con1", 0),
            output_Q=("con1", 0),
            input_I=("con1", 0),
            input_Q=("con1", 0),
        )
    },
)

machine.wiring = wiring

## Add octaves

In [32]:
from squid_lab_quam.components.octave import OctaveSQuID

octaves = {
    "octave1": OctaveSQuID(
        ip=network.octave_networks["octave1"].get_reference("octave_host"),
        port=network.octave_networks["octave1"].get_reference("octave_port"),
    )
}
machine.octaves = octaves

## Add readout frequency converters

In [33]:
from squid_lab_quam.components.octave import (
    OctaveUpConverterSQuID,
    OctaveDownConverterSQuID,
)


readout_up_converter = OctaveUpConverterSQuID(LO_frequency=5e9)


octaves["octave1"].RF_outputs[1] = readout_up_converter


readout_down_converter = OctaveDownConverterSQuID(
    LO_frequency=readout_up_converter.get_reference("LO_frequency"),
)


octaves["octave1"].RF_inputs[1] = readout_down_converter

## Add qubits

In [41]:
from squid_lab_quam.components.qubits import Transmon
from squid_lab_quam.components.resonators import ReadoutResonator
from quam.components.channels import IQChannel, InOutIQChannel
from squid_lab_quam.components.pulse_sets import PulseSetDragGaussian, PulseSetFlattopCosine
from quam.components.pulses import SquareReadoutPulse

qubit_names = ["q0", "q1", "q2", "q3", "q4"]

machine.shared_qubit_parameters = {
    "drag_gaussian_pulse_parameters": {"length": 40, "sigma": 7, "subtracted": True},
}

for idx, qubit_name in enumerate(qubit_names):
    # Add driving frequency converter
    drive_up_converter = OctaveUpConverterSQuID(LO_frequency=5.5e9)
    octaves["octave1"].RF_outputs[idx + 2] = drive_up_converter

    # Add qubit
    qubit = Transmon(
        xy=IQChannel(
            opx_output_I=f"#/wiring/drive_lines/{qubit_name}/port_I",
            opx_output_Q=f"#/wiring/drive_lines/{qubit_name}/port_Q",
            frequency_converter_up=drive_up_converter.get_reference(),
        ),
        resonator=ReadoutResonator(
            channel=InOutIQChannel(
                opx_input_I=f"#/wiring/feed_lines/feedline/input_I",
                opx_input_Q=f"#/wiring/feed_lines/feedline/input_Q",
                opx_output_I=f"#/wiring/feed_lines/feedline/output_I",
                opx_output_Q=f"#/wiring/feed_lines/feedline/output_Q",
                frequency_converter_up=readout_up_converter.get_reference(),
                frequency_converter_down=readout_down_converter.get_reference(),
            )
        ),
        transition_frequencies=[6, 5],
    )
    qubit.pulse_sets = {
        "drag_gaussian": PulseSetDragGaussian(
            amplitude_180=0.5,
            amplitude_90=0.25,
            sigma="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/sigma",
            length="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/length",
            subtracted="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/subtracted",
            anharmonicity="#../../anharmonicity",
        ),
        "flattop_cosine": PulseSetFlattopCosine(
            amplitude=0.1,
            rise_fall_time=16,
        )
    }
    

    machine.qubits[qubit_name] = qubit
    qubit.pulse_sets["drag_gaussian"].add_drive_pulses()
    qubit.pulse_sets["flattop_cosine"].add_drive_pulses()


    qubit.resonator.channel.operations["readout"] = SquareReadoutPulse(
        length=1000, amplitude=0.1, threshold=0.0, digital_marker="ON"
    )

In [40]:
machine.qubits["q0"].xy.frequency_converter_up.LO_frequency 

In [36]:
machine.qubits["q0"]

c:\Users\drn455\AppData\Local\anaconda3\envs\opx_control\Lib\site-packages\quam\core\quam_classes.py:487: UserWarning: Could not get reference #./inferred_RF_frequency from IQChannel: #/qubits/q0/xy.
String #./inferred_RF_frequency is not a valid reference, Error: Error inferring RF frequency for channel q0.xy: LO_frequency is not a number: None
  warnings.warn(f"Could not get reference {reference} from {ref}.\n{str(e)}")


Transmon(id='q0', xy=IQChannel(operations={'x90_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marker=None, axis_angle=0, amplitude=0.25, sigma=7, alpha=0, anharmonicity=-1, detuning=0, subtracted=True), 'x180_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marker=None, axis_angle=0, amplitude=0.5, sigma=7, alpha=0, anharmonicity=-1, detuning=0, subtracted=True), 'y90_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marker=None, axis_angle=90, amplitude=0.25, sigma=7, alpha=0, anharmonicity=-1, detuning=0, subtracted=True), 'y180_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marker=None, axis_angle=90, amplitude=0.5, sigma=7, alpha=0, anharmonicity=-1, detuning=0, subtracted=True), '-x90_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marker=None, axis_angle=0, amplitude=-0.25, sigma=7, alpha=0, anharmonicity=-1, detuning=0, subtracted=True), '-y90_drag_gaussian': DragGaussianPulse(length=40, id=None, digital_marke

In [37]:
machine.save()

In [38]:
machine_loaded = QuAMSCQ1.load(machine.information.state_path)

In [39]:
machine_loaded.qubits["q0"].pulse_sets["flattop_cosine"].gates

KeyError: 'flattop_cosine'

In [ ]:
import os

In [ ]:
os.path.exists(machine_loaded.information.data_path)

True